# <center>Создание декоратора класса

Представьте себя ненадолго разработчиком компьютерной игры в стиле фэнтези. Вы будете прописывать систему эффектов, которые могут быть наложены на героя вашей игры.

У вас есть герой, который обладает некоторым набором характеристик. Враги и союзники могут накладывать на героя положительные и отрицательные эффекты. Эти эффекты каким-то образом изменяют характеристики героя. На героя можно накладывать бесконечно много эффектов, действие одинаковых эффектов суммируется. Игрок должен знать, какие положительные и какие отрицательные эффекты на него были наложены и в каком порядке.

Класс герой описан следующим образом (характеристики могут быть другими):

In [ ]:
from abc import ABC, abstractmethod

In [2]:
class Hero:
    """
      Абстрактный объект (Component)
    """
    def __init__(self):
        self.positive_effects = []
        self.negative_effects = []
        
        self.stats = {
            "HP": 128,
            "MP": 42,
            "SP": 100,
            
            "Strength": 15,
            "Perception": 4,
            "Endurance": 8,
            "Charisma": 2,
            "Intelligence": 3,
            "Agility": 8,
            "Luck": 1
        } 
        
    def get_positive_effects(self):
        return self.positive_effects.copy()
    
    def get_negative_effects(self):
        return self.negative_effects.copy()
    
    def get_stats(self):
        return self.stats.copy()
        

Описывать класс героя в коде НЕ НУЖНО.

Вам нужно написать систему декораторов, представленную на UML-диаграмме:

<img src="images/cloudfront_net.jpg">

Названия наложенных положительных и отрицательных эффектов добавляются каждое в свой счетчик. Названия эффектов совпадают с названиями классов.

Описания эффектов:

- **Берсерк (Berserk)** — Увеличивает параметры *Сила, Выносливость, Ловкость, Удача* на 7; уменьшает параметры *Восприятие, Харизма, Интеллект* на 3. Количество единиц здоровья увеличивается на 50.
- **Благословение (Blessing)** — Увеличивает все основные характеристики на 2.
- **Слабость (Weakness)** — Уменьшает параметры *Сила, Выносливость, Ловкость* на 4.
- **Сглаз (EvilEye)** — Уменьшает параметр *Удача* на 10.
- **Проклятье (Curse)** — Уменьшает все основные характеристики на 2.   

К основным характеристикам относятся Сила (`Strength`), Восприятие (`Perception`), Выносливость (`Endurance`), Харизма (`Charisma`), Интеллект (`Intelligence`), Ловкость (`Agility`), Удача (`Luck`).

При выполнении задания учитывайте, что:

- Изначальные характеристики базового объекта не должны меняться.
- Изменения характеристик и накладываемых эффектов (баффов/дебаффов) должно происходить **динамически**, то есть при запросе `get_stats`, `get_positive_effects`, `get_negative_effects`
- Абстрактные классы AbstractPositive, AbstractNegative и соответственно их потомки могут принимать любой параметр base при инициализации объекта (`__ init__ (self, base)`)
- Проверяйте, что эффекты корректно снимаются, в том числе и из середины стека

In [3]:
class AbstractEffect(Hero, ABC):
    """
    Абстрактный декоратор (Decorator)
    """
    def __init__(self, base):
        super().__init__()
        self.base = base
        self.base_characteristics = ["Strength", "Perception", "Endurance", "Charisma", "Intelligence", "Agility", "Luck"]
        
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта 
        stats = self.base.get_stats()
        return stats
    
    def get_positive_effects(self):
        return self.base.get_positive_effects()
    
    def get_negative_effects(self):
        return self.base.get_negative_effects()

    
class AbstractPositive(AbstractEffect):
    """
    Абстрактный декоратор (Decorator) (more less level)
    """
    @abstractmethod
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта 
        stats = self.base.get_stats()
        return stats
    
    @abstractmethod
    def get_positive_effects(self):
        return self.base.get_positive_effects()

    
class AbstractNegative(AbstractEffect):
    """
    Абстрактный декоратор (Decorator) (more less level)
    """
    @abstractmethod
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта 
        stats = self.base.get_stats()
        return stats
    
    @abstractmethod
    def get_negative_effects(self):
        return self.base.get_negative_effects()

class Berserk(AbstractPositive):
    """
    Конкретный декоратор
    """
        
    def _berserk(self, stats):
        for char in ["Strength", "Endurance", "Agility", "Luck"]:
            stats[char] += 7
        for char in ["Perception", "Charisma", "Intelligence"]:
            stats[char] -= 3
        stats["HP"] += 50
        return stats
        
    def get_positive_effects(self):
        positive_effects = self.base.get_positive_effects()
        positive_effects.append("Berserk")
        return positive_effects
    
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта
        stats = self.base.get_stats()
        return self._berserk(stats)

class Blessing(AbstractPositive):
    """
    Конкретный декоратор
    """
        
    def _blessing(self, stats):
        for char in self.base_characteristics:
            stats[char] += 2
        return stats
            
    def get_positive_effects(self):
        positive_effects = self.base.get_positive_effects()
        positive_effects.append("Blessing")
        return positive_effects
    
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта
        stats = self.base.get_stats()
        return self._blessing(stats)

class Weakness(AbstractNegative):
    """
    Конкретный декоратор
    """
        
    def _weakness(self, stats):
        for char in ["Strength", "Endurance", "Agility"]:
            stats[char] -= 4
        return stats
            
    def get_negative_effects(self):
        negative_effects = self.base.get_negative_effects()
        negative_effects.append("Weakness")
        return negative_effects
    
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта
        stats = self.base.get_stats()
        return self._weakness(stats)

class EvilEye(AbstractNegative):
    """
    Конкретный декоратор
    """
        
    def _evil_eye(self, stats):
        for char in ["Luck"]:
            stats[char] -= 10
        return stats
            
    def get_negative_effects(self):
        negative_effects = self.base.get_negative_effects()
        negative_effects.append("EvilEye")
        return negative_effects
    
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта
        stats = self.base.get_stats()
        return self._evil_eye(stats)
    

class Curse(AbstractNegative):
    """
    Конкретный декоратор
    """
        
    def _curse(self, stats):
        for char in self.base_characteristics:
            stats[char] -= 2
        return stats
            
    def get_negative_effects(self):
        negative_effects = self.base.get_negative_effects()
        negative_effects.append("Curse")
        return negative_effects
    
    def get_stats(self): # Возвращает итоговые характеристики после применения эффекта
        stats = self.base.get_stats()
        return self._curse(stats)

In [74]:
hero = Hero()

hero1 = Curse(hero)
hero2 = EvilEye(hero1)
hero3 = Berserk(hero2)
hero4 = Weakness(hero3)

In [69]:
hero4.get_negative_effects()

['Curse', 'EvilEye', 'Weakness']

In [70]:
hero4.get_positive_effects()

['Berserk']

In [71]:
hero3.base = hero3.base.base

In [75]:
hero4.base = hero4.base.base

In [76]:
hero4.get_negative_effects()

['Curse', 'EvilEye', 'Weakness']

In [77]:
hero4.get_positive_effects()

[]